## Ejemplo de uso

Vamos a usar el modelo para hacer predicciones sobre una muestra de las facturas. Convertiremos las etiquetas a un diccionario JSON y complementaremos posibles fallos del modelo con reglas.








In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install seqeval
!pip install accelerate -U
!pip install evaluate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylin

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, Features, Sequence, ClassLabel, Value
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import numpy as np
from google.colab import drive
import evaluate
import re
from collections import Counter
from heapq import nlargest
from datetime import datetime



In [ ]:
# Configurar el nivel de verbosidad del logging
from transformers import logging
logging.set_verbosity_error()

# Montar Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


Usamos un dataset sobre el que no se ha aplicado aumento de datos. Como vamos a hacer una simulación del uso del modelo, el objeto datasets en esta ocasión no tiene etiquetas NER.






In [ ]:
# Cargar el DataFrame
csv_path = "/content/drive/MyDrive/dataset_facturas_etiquetado_2.csv"
df = pd.read_csv(csv_path)

# Crear el DataFrame sin etiquetas NER
df_unlabeled = df.loc[:, ['id', 'texto']]
df_unlabeled.rename(columns={'texto': 'tokens'}, inplace=True)

# Convertir las cadenas de tokens a listas
def to_list(column):
    return eval(column)

df_unlabeled['tokens'] = df_unlabeled['tokens'].apply(to_list)


In [ ]:
# Convertir a Dataset de HuggingFace
dataset_unlabeled = Dataset.from_pandas(df_unlabeled)

if '__index_level_0__' in dataset_unlabeled.column_names:
    dataset_unlabeled = dataset_unlabeled.remove_columns(['__index_level_0__'])

features_unlabeled = Features({
    'id': Value(dtype='string'),
    'tokens': Sequence(Value(dtype='string'))
})

dataset_unlabeled = dataset_unlabeled.cast(features_unlabeled)

Casting the dataset:   0%|          | 0/914 [00:00<?, ? examples/s]

Tokenizamos y realizamos las predicciones.

In [ ]:
# Definir la lista de etiquetas NER
label_list = ['O', 'B-NOM', 'I-NOM', 'B-DNI', 'I-DNI', 'B-CAL', 'I-CAL', 'B-CP', 'I-CP', 'B-LOC', 'I-LOC', 'B-PRO', 'I-PRO',
              'B-NOMC', 'I-NOMC', 'B-CIF', 'I-CIF', 'B-DIRC', 'I-DIRC', 'B-CPC', 'I-CPC', 'B-LOCC', 'I-LOCC', 'B-PROC', 'I-PROC',
              'B-NUMF', 'I-NUMF', 'B-INI', 'I-INI', 'B-FIN', 'I-FIN', 'B-FAC', 'I-FAC', 'B-CAR','I-CAR', 'B-PER',
              'I-PER', 'B-POT', 'I-POT']

In [ ]:
# Cargar el tokenizer y el modelo guardado
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
model_path = "/content/drive/MyDrive/model_15_035_def"
model = AutoModelForTokenClassification.from_pretrained(model_path, num_labels=len(label_list))

# Tokenizar y alinear las etiquetas
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)
    return tokenized_inputs

tokenized_dataset_unlabeled = dataset_unlabeled.map(tokenize_and_align_labels, batched=True)

# Definir el data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Configurar el Trainer para predicción
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/output',  # Directorio donde se guardarán los resultados
    per_device_eval_batch_size=16,
    logging_dir='/content/drive/MyDrive/logs',  # Directorio para guardar los logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Realizar predicciones en el conjunto no etiquetado
predictions, _, _ = trainer.predict(tokenized_dataset_unlabeled)
predictions = np.argmax(predictions, axis=2)

# Convertir las predicciones a nombres de etiquetas
true_predictions = [
    [label_list[p] for p in prediction]
    for prediction in predictions
]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

Volvemos a convertir el objeto datasets a un DataFrame y le agregamos la columna "predict".








In [ ]:
# Convertir tokenized_dataset_unlabeled a DataFrame
unlabeled_df = tokenized_dataset_unlabeled.to_pandas()

# Asegurarse de que las columnas necesarias están presentes en el DataFrame
if 'input_ids' in unlabeled_df.columns:
    unlabeled_df['tokens'] = unlabeled_df['input_ids'].apply(lambda x: tokenizer.convert_ids_to_tokens(x))

# Agregar las predicciones (true_predictions) al DataFrame
unlabeled_df['predict'] = true_predictions

# Mostrar las primeras filas del DataFrame para verificar
unlabeled_df.head()

# Guardar los resultados en un nuevo archivo CSV
#output_path = "/content/drive/MyDrive/dataset_con_predicciones.csv"
#unlabeled_df.to_csv(output_path, index=False)

,id,tokens,input_ids,token_type_ids,attention_mask,predict
0,factura_836,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, IM...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,factura_264,"[[CLS], L, ., ., ., u, ., ., n, ., ., e, ., .,...","[4, 1078, 1009, 1009, 1009, 1553, 1009, 1009, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,factura_533,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU..."
3,factura_311,"[[CLS], L, ., ., ., u, ., ., n, ., ., e, ., .,...","[4, 1078, 1009, 1009, 1009, 1553, 1009, 1009, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,factura_473,"[[CLS], L, ., ., ., u, ., ., n, ., ., e, ., .,...","[4, 1078, 1009, 1009, 1009, 1553, 1009, 1009, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
# Crear una nueva columna 'id_num' con los números extraídos de la columna 'id'
unlabeled_df['id_num'] = unlabeled_df['id'].apply(lambda x: int(x.split('_')[-1]))

# Ordenar el DataFrame por la nueva columna 'id_num'
unlabeled_df = unlabeled_df.sort_values(by='id_num')

# Eliminar la columna 'id_num' si ya no es necesaria
unlabeled_df = unlabeled_df.drop(columns=['id_num'])

# Reseteamos el índice
unlabeled_df = unlabeled_df.reset_index()


In [ ]:
unlabeled_df

,index,id,tokens,input_ids,token_type_ids,attention_mask,predict
0,897,factura_0,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU..."
1,744,factura_2,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU..."
2,607,factura_3,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU..."
3,360,factura_4,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, IM...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,421,factura_5,"[[CLS], L, ., ., ., u, ., ., n, ., ., e, ., .,...","[4, 1078, 1009, 1009, 1009, 1553, 1009, 1009, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...,...,...,...
909,816,factura_994,"[[CLS], Página, 1, /, 2, VI, ##R, ##G, ##IN, E...","[4, 16644, 1094, 972, 1115, 3903, 30980, 30992...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, B-NOMC, B-NOMC, B-NOMC, B-NOMC..."
910,304,factura_995,"[[CLS], Página, 1, /, 2, EL, ##EC, ##TR, ##ICA...","[4, 16644, 1094, 972, 1115, 4263, 4142, 4662, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, B-NOMC, B-NOMC, B-NOMC, B-NOMC..."
911,554,factura_996,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU..."
912,313,factura_997,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, IM...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
"""# Función para eliminar tokens específicos de una lista de tokens
def remove_special_tokens(tokens):
    return [token for token in tokens if token not in ['[CLS]', '[SEP]']]

# Aplicar la función a la columna tokens
unlabeled_df['tokens'] = unlabeled_df['tokens'].apply(remove_special_tokens)

unlabeled_df"""

"# Función para eliminar tokens específicos de una lista de tokens\ndef remove_special_tokens(tokens):\n    return [token for token in tokens if token not in ['[CLS]', '[SEP]']]\n\n# Aplicar la función a la columna tokens\nunlabeled_df['tokens'] = unlabeled_df['tokens'].apply(remove_special_tokens)\n\nunlabeled_df"

In [ ]:
# Verificar filas con longitudes desiguales en 'tokens' y 'predict'
filas_con_errores = []

for index, row in unlabeled_df.iterrows():
    if len(row['tokens']) != len(row['predict']):
        filas_con_errores.append(index)

# Mostrar las filas con errores
print("Filas con longitudes desiguales en 'tokens' y 'predict':", filas_con_errores)

Filas con longitudes desiguales en 'tokens' y 'predict': []


In [ ]:
def unir_puntuacion(lista_palabras):
    palabras_originales = []
    buffer = ''

    for palabra in lista_palabras:
        if palabra.isalnum():
            if buffer:
                if palabras_originales:
                    palabras_originales[-1] += buffer
                else:
                    palabras_originales.append(buffer)
                buffer = ''
            palabras_originales.append(palabra)
        else:
            buffer += palabra

    if buffer:
        if palabras_originales:
            palabras_originales[-1] += buffer
        else:
            palabras_originales.append(buffer)

    return palabras_originales




Ahora vamos a usar la función comprobar_etiquetas para extraer las etiquetas que ha predicho el modelo a partir del texto.

In [ ]:
# Agregar una columna 'json_comprobacion' con diccionario vacío
unlabeled_df['json_comprobacion'] = [{} for _ in range(len(unlabeled_df))]

In [ ]:
def comprobar_etiquetas(df):
    for index, row in df.iterrows():
        # Inicializar listas para almacenar las ocurrencias de las diferentes entidades como cadena
        ocurrencias_nombre_cliente = []
        ocurrencias_dni_cliente = []
        ocurrencias_calle_cliente = []
        ocurrencias_cp_cliente = []
        ocurrencias_población_cliente = []
        ocurrencias_provincia_cliente = []
        ocurrencias_nombre_comercializadora = []
        ocurrencias_cif_comercializadora = []
        ocurrencias_dirección_comercializadora = []
        ocurrencias_cp_comercializadora = []
        ocurrencias_población_comercializadora = []
        ocurrencias_provincia_comercializadora = []
        ocurrencias_número_factura = []
        ocurrencias_inicio_periodo = []
        ocurrencias_fin_periodo = []
        ocurrencias_importe_factura = []
        ocurrencias_fecha_cargo = []
        ocurrencias_consumo_periodo = []
        ocurrencias_potencia_contratada = []

        # Inicializar listas para almacenar las palabras de las diferentes entidades
        nombre_cliente = []
        dni_cliente = []
        calle_cliente = []
        cp_cliente = []
        población_cliente = []
        provincia_cliente = []
        nombre_comercializadora = []
        cif_comercializadora = []
        dirección_comercializadora = []
        cp_comercializadora = []
        población_comercializadora = []
        provincia_comercializadora = []
        número_factura = []
        inicio_periodo = []
        fin_periodo = []
        importe_factura = []
        fecha_cargo = []
        consumo_periodo = []
        potencia_contratada = []

        # Iterar sobre las etiquetas para buscar secuencias correspondientes a diferentes entidades
        for i, etiqueta in enumerate(row['predict']):
            if etiqueta == 'B-NOM':
                # Reiniciar la lista de palabras del nombre del cliente para cada nueva ocurrencia
                nombre_cliente = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-NOM' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-NOM':
                        # Agregar la palabra al nombre del cliente
                        nombre_cliente.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NOM'
                # Agregar la ocurrencia del nombre del cliente como cadena a la lista
                ocurrencias_nombre_cliente.append(' '.join(unir_puntuacion(nombre_cliente)))

            elif etiqueta == 'B-DNI':
                # Reiniciar la lista de palabras del DNI del cliente para cada nueva ocurrencia
                dni_cliente = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-DNI' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-DNI':
                        # Agregar la palabra al DNI del cliente
                        dni_cliente.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-DNI'
                # Agregar la ocurrencia del DNI del cliente como cadena a la lista
                ocurrencias_dni_cliente.append(' '.join(unir_puntuacion(dni_cliente)))

            elif etiqueta == 'B-CAL':
                # Reiniciar la lista de palabras de la calle del cliente para cada nueva ocurrencia
                calle_cliente = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-CAL' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-CAL':
                        # Agregar la palabra a la calle del cliente
                        calle_cliente.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CAL'
                # Agregar la ocurrencia de la calle del cliente como cadena a la lista
                ocurrencias_calle_cliente.append(' '.join(unir_puntuacion(calle_cliente)))

            elif etiqueta == 'B-CP':
                # Reiniciar la lista de palabras del código postal del cliente para cada nueva ocurrencia
                cp_cliente = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-CP' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-CP':
                        # Agregar la palabra al código postal del cliente
                        cp_cliente.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CP'
                # Agregar la ocurrencia del código postal del cliente como cadena a la lista
                ocurrencias_cp_cliente.append(' '.join(unir_puntuacion(cp_cliente)))

            elif etiqueta == 'B-LOC':
                # Reiniciar la lista de palabras de la población del cliente para cada nueva ocurrencia
                población_cliente = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-LOC' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-LOC':
                        # Agregar la palabra a la población del cliente
                        población_cliente.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-LOC'
                # Agregar la ocurrencia de la población del cliente como cadena a la lista
                ocurrencias_población_cliente.append(' '.join(unir_puntuacion(población_cliente)))

            elif etiqueta == 'B-PRO':
                provincia_cliente = [row['tokens'][i]]
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-PRO':
                        provincia_cliente.append(row['tokens'][j])
                    else:
                        break
                ocurrencias_provincia_cliente.append(' '.join(unir_puntuacion(provincia_cliente)))

            elif etiqueta == 'B-NOMC':
                # Reiniciar la lista de palabras del nombre de la comercializadora para cada nueva ocurrencia
                nombre_comercializadora = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-NOMC' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-NOMC':
                        # Agregar la palabra al nombre de la comercializadora
                        nombre_comercializadora.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NOMC'
                # Agregar la ocurrencia del nombre de la comercializadora como cadena a la lista
                ocurrencias_nombre_comercializadora.append(' '.join(unir_puntuacion(nombre_comercializadora)))

            elif etiqueta == 'B-CIF':
                # Reiniciar la lista de palabras del CIF de la comercializadora para cada nueva ocurrencia
                cif_comercializadora = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-CIF' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-CIF':
                        # Agregar la palabra al CIF de la comercializadora
                        cif_comercializadora.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CIF'
                # Agregar la ocurrencia del CIF de la comercializadora como cadena a la lista
                ocurrencias_cif_comercializadora.append(' '.join(unir_puntuacion(cif_comercializadora)))

            elif etiqueta == 'B-DIRC':
                # Reiniciar la lista de palabras de la dirección de la comercializadora para cada nueva ocurrencia
                dirección_comercializadora = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-DIRC' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-DIRC':
                        # Agregar la palabra a la dirección de la comercializadora
                        dirección_comercializadora.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-DIRC'
                # Agregar la ocurrencia de la dirección de la comercializadora como cadena a la lista
                ocurrencias_dirección_comercializadora.append(' '.join(unir_puntuacion(dirección_comercializadora)))

            elif etiqueta == 'B-CPC':
                # Reiniciar la lista de palabras del código postal de la comercializadora para cada nueva ocurrencia
                cp_comercializadora = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-CPC' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-CPC':
                        # Agregar la palabra al código postal de la comercializadora
                        cp_comercializadora.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CPC'
                # Agregar la ocurrencia del código postal de la comercializadora como cadena a la lista
                ocurrencias_cp_comercializadora.append(' '.join(unir_puntuacion(cp_comercializadora)))

            elif etiqueta == 'B-LOCC':
                # Reiniciar la lista de palabras de la población de la comercializadora para cada nueva ocurrencia
                población_comercializadora = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-LOCC' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-LOCC':
                        # Agregar la palabra a la población de la comercializadora
                        población_comercializadora.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-LOCC'
                # Agregar la ocurrencia de la población de la comercializadora como cadena a la lista
                ocurrencias_población_comercializadora.append(' '.join(unir_puntuacion(población_comercializadora)))

            elif etiqueta == 'B-PROC':
                provincia_comercializadora = [row['tokens'][i]]
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-PROC':
                        provincia_comercializadora.append(row['tokens'][j])
                    else:
                        break
                ocurrencias_provincia_comercializadora.append(' '.join(unir_puntuacion(provincia_comercializadora)))

            elif etiqueta == 'B-NUMF':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                número_factura = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-NUMF' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-NUMF':
                        # Agregar la palabra al número de factura
                        número_factura.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_número_factura.append(' '.join(unir_puntuacion(número_factura)))

            elif etiqueta == 'B-INI':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                inicio_periodo = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-INI' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-INI':
                        # Agregar la palabra al número de factura
                        inicio_periodo.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-INI'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_inicio_periodo.append(' '.join(unir_puntuacion(inicio_periodo)))

            elif etiqueta == 'B-FIN':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                fin_periodo = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-FIN' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-FIN':
                        # Agregar la palabra al número de factura
                        fin_periodo.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-FIN'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_fin_periodo.append(' '.join(unir_puntuacion(fin_periodo)))

            elif etiqueta == 'B-FAC':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                importe_factura = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-FAC' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-FAC':
                        # Agregar la palabra al número de factura
                        importe_factura.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_importe_factura.append(' '.join(unir_puntuacion(importe_factura)))

            elif etiqueta == 'B-CAR':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                fecha_cargo = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-NUMF' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-CAR':
                        # Agregar la palabra al número de factura
                        fecha_cargo.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_fecha_cargo.append(' '.join(unir_puntuacion(fecha_cargo)))

            elif etiqueta == 'B-PER':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                consumo_periodo = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-PER' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-PER':
                        # Agregar la palabra al número de factura
                        consumo_periodo.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-PER'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_consumo_periodo.append(' '.join(unir_puntuacion(consumo_periodo)))

            elif etiqueta == 'B-POT':
                # Reiniciar la lista de palabras de potencia_contatada para cada nueva ocurrencia
                potencia_contratada = [row['tokens'][i]]
                # Buscar si hay etiquetas 'I-POT' después
                for j in range(i+1, len(row['predict'])):
                    if row['predict'][j] == 'I-POT':
                        # Agregar la palabra al número de factura
                        potencia_contratada.append(row['tokens'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-POT'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_potencia_contratada.append(' '.join(unir_puntuacion(potencia_contratada)))

        # Agregar las ocurrencias al DataFrame bajo la columna 'json_comprobacion'
        df.at[index, 'json_comprobacion']['nombre_cliente'] = ocurrencias_nombre_cliente
        df.at[index, 'json_comprobacion']['dni_cliente'] = ocurrencias_dni_cliente
        df.at[index, 'json_comprobacion']['calle_cliente'] = ocurrencias_calle_cliente
        df.at[index, 'json_comprobacion']['cp_cliente'] = ocurrencias_cp_cliente
        df.at[index, 'json_comprobacion']['población_cliente'] = ocurrencias_población_cliente
        df.at[index, 'json_comprobacion']['provincia_cliente'] = ocurrencias_provincia_cliente  # Agregar provincia_cliente
        df.at[index, 'json_comprobacion']['nombre_comercializadora'] = ocurrencias_nombre_comercializadora
        df.at[index, 'json_comprobacion']['cif_comercializadora'] = ocurrencias_cif_comercializadora
        df.at[index, 'json_comprobacion']['dirección_comercializadora'] = ocurrencias_dirección_comercializadora
        df.at[index, 'json_comprobacion']['cp_comercializadora'] = ocurrencias_cp_comercializadora
        df.at[index, 'json_comprobacion']['población_comercializadora'] = ocurrencias_población_comercializadora
        df.at[index, 'json_comprobacion']['provincia_comercializadora'] = ocurrencias_provincia_comercializadora  # Agregar provincia_comercializadora
        df.at[index, 'json_comprobacion']['número_factura'] = ocurrencias_número_factura
        df.at[index, 'json_comprobacion']['inicio_periodo'] = ocurrencias_inicio_periodo
        df.at[index, 'json_comprobacion']['fin_periodo'] = ocurrencias_fin_periodo
        df.at[index, 'json_comprobacion']['importe_factura'] = ocurrencias_importe_factura
        df.at[index, 'json_comprobacion']['fecha_cargo'] = ocurrencias_fecha_cargo
        df.at[index, 'json_comprobacion']['consumo_periodo'] = ocurrencias_consumo_periodo
        df.at[index, 'json_comprobacion']['potencia_contratada'] = ocurrencias_potencia_contratada


    return df

In [ ]:
comprobar_etiquetas(unlabeled_df)

,index,id,tokens,input_ids,token_type_ids,attention_mask,predict,json_comprobacion
0,897,factura_0,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...","{'nombre_cliente': ['Conrad', '##o Daniel Igle..."
1,744,factura_2,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...","{'nombre_cliente': ['BE', '##NE', '##DI', '##C..."
2,607,factura_3,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...","{'nombre_cliente': ['Bel', '##inda Z##etina Mi..."
3,360,factura_4,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, IM...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': ['PAN', '##TA', '##LE', '##..."
4,421,factura_5,"[[CLS], L, ., ., ., u, ., ., n, ., ., e, ., .,...","[4, 1078, 1009, 1009, 1009, 1553, 1009, 1009, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': ['SA', '##TUR', '##NI', '##..."
...,...,...,...,...,...,...,...,...
909,816,factura_994,"[[CLS], Página, 1, /, 2, VI, ##R, ##G, ##IN, E...","[4, 16644, 1094, 972, 1115, 3903, 30980, 30992...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, B-NOMC, B-NOMC, B-NOMC, B-NOMC...","{'nombre_cliente': ['PE', '##TR', '##ON', '##I..."
910,304,factura_995,"[[CLS], Página, 1, /, 2, EL, ##EC, ##TR, ##ICA...","[4, 16644, 1094, 972, 1115, 4263, 4142, 4662, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, B-NOMC, B-NOMC, B-NOMC, B-NOMC...","{'nombre_cliente': ['SU', '##LP', '##ICI', '##..."
911,554,factura_996,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, [U...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...","{'nombre_cliente': ['Petro', '##na Ur##ib##e N..."
912,313,factura_997,"[[CLS], DA, ##TOS, DE, LA, FA, ##C, ##TURA, IM...","[4, 15239, 12121, 1982, 3655, 11488, 30961, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': ['CE', '##LE', '##ST', '##I..."


In [ ]:
unlabeled_df['json_comprobacion'][4]

{'nombre_cliente': ['SA',
  '##TUR',
  '##NI',
  '##NO MA##L##T##É##S NA##RA##N##J##O'],
 'dni_cliente': ['13', '##7', '##14', '##12', '##2', '##G'],
 'calle_cliente': ['Calle de la Fuente de Ant##ón Mer##lo',
  'Calle de la Fuente de Ant##ón Mer##lo'],
 'cp_cliente': ['249', '##20', '249', '##20'],
 'población_cliente': ['Val',
  '##de',
  '##pol',
  '##o',
  'Val',
  '##de',
  '##pol',
  '##o'],
 'provincia_cliente': [],
 'nombre_comercializadora': ['EN',
  '##ERV',
  '##ER EN##ER##G##IA, S. L.',
  'EN',
  '##ERV',
  '##ER EN##ER##G##IA, S. L.'],
 'cif_comercializadora': [],
 'dirección_comercializadora': ['CAL', '##LE 42, POL##IG##ON##O EL BO##N##Y'],
 'cp_comercializadora': ['46', '##7', '##40'],
 'población_comercializadora': [],
 'provincia_comercializadora': ['V', '##AL', '##ENCIA'],
 'número_factura': ['K', '##S', '##86', '##22', '##02', '##28', '##48'],
 'inicio_periodo': [],
 'fin_periodo': [],
 'importe_factura': [],
 'fecha_cargo': [],
 'consumo_periodo': [],
 'potencia_con


Los valores del diccionario tienen este aspecto debido a los efectos de la tokenización. La siguiente función los reconstruye.

In [ ]:
def reconstruct_entities_combined(dictionary):
    reconstructed_dict = {}

    for key, entities in dictionary.items():
        # Paso 1: Eliminar "[UNK]" y reconstruir entidades basadas en "##"
        reconstructed = []
        current_entity = ""

        for entity in entities:
            # Eliminar los "[UNK]"
            entity = entity.replace("[UNK]", "")

            if "##" in entity:
                entity = entity.replace("##", "")
                current_entity += entity
            elif any(punctuation in entity for punctuation in ["-", ",", ".", ":", ";", "!", "?", "/"]):
                current_entity += entity
            else:
                if current_entity:
                    reconstructed.append(current_entity)
                current_entity = entity

        if current_entity:
            reconstructed.append(current_entity)

        # Paso 2: Verificar y reconstruir cadenas que terminan en signos de puntuación
        entities = reconstructed
        reconstructed = []
        while entities:
            if entities[-1].endswith(("-", ",", ".", ":", ";", "!", "?", "/")):
                reconstructed.append(entities.pop())
            else:
                entity = entities.pop()
                while entities and entities[-1].endswith(("-", ",", ".", ":", ";", "!", "?", "/")):
                    entity = entities.pop() + entity
                reconstructed.append(entity)

        reconstructed.reverse()

        # Paso 3: Reconstruir entidades basadas en las últimas seis letras repetidas
        entities = reconstructed
        reconstructed = []

        for entity in entities:
            last_six = entity[-6:]
            if entity.count(last_six) > 1:
                first_occurrence_index = entity.index(last_six)
                reconstructed.append(entity[:first_occurrence_index + len(last_six)])
            else:
                reconstructed.append(entity)

        reconstructed_dict[key] = reconstructed

    return reconstructed_dict


In [ ]:
unlabeled_df['json_comprobacion'] = unlabeled_df['json_comprobacion'].apply(reconstruct_entities_combined)


In [ ]:
unlabeled_df['json_comprobacion'][1]

{'nombre_cliente': ['BENEDICTA GALLEGOS AGUILAR'],
 'dni_cliente': [],
 'calle_cliente': ['Travesia de las Delicias'],
 'cp_cliente': ['24149'],
 'población_cliente': ['Los Barrios de Luna'],
 'provincia_cliente': ['León'],
 'nombre_comercializadora': [],
 'cif_comercializadora': ['A11508629'],
 'dirección_comercializadora': ['C/ CARRETAS, 5'],
 'cp_comercializadora': ['11510'],
 'población_comercializadora': ['PUERTO REAL'],
 'provincia_comercializadora': [],
 'número_factura': ['H4623704265'],
 'inicio_periodo': ['13/11/1999'],
 'fin_periodo': ['13/12/1999'],
 'importe_factura': [],
 'fecha_cargo': ['18 de diciembre de 1999', '18 de diciembre de 1999'],
 'consumo_periodo': [],
 'potencia_contratada': []}

Como puede haber varias apariciones de cada valor en el diccionario, o incluso distintas (aunque esto se debe a errores en la predicción), la siguiente función se encarga de elegir uno.

In [ ]:
def select_longest_string(d):
    # Iterar sobre el diccionario
    for key, value in d.items():
        # Si el valor es una lista de strings
        if isinstance(value, list) and all(isinstance(item, str) for item in value):
            # Seleccionar el string más largo y, en caso de empate, el primero en la lista
            longest_string = max(value, key=lambda x: (len(x), value.index(x)), default=None)
            # Actualizar el valor de la clave con el string más largo
            d[key] = [longest_string] if longest_string else []
    return d

# Aplicar la función select_longest_string a la columna 'json_comprobacion' del DataFrame
unlabeled_df['json_comprobacion'] = unlabeled_df['json_comprobacion'].apply(select_longest_string)


In [ ]:
unlabeled_df['json_comprobacion'][897]

{'nombre_cliente': ['PIERA GALÁN MONTALVO'],
 'dni_cliente': [],
 'calle_cliente': ['Calle de Molina'],
 'cp_cliente': ['09227'],
 'población_cliente': ['Villaldemiro'],
 'provincia_cliente': ['Burgos'],
 'nombre_comercializadora': ['LOOP ELECTRICIDAD Y GAS, S. L'],
 'cif_comercializadora': ['B87095543'],
 'dirección_comercializadora': ['C/ CARDENAL MARCELO SPÍNOLA, 8, DERECHA'],
 'cp_comercializadora': ['28023'],
 'población_comercializadora': ['MADRID'],
 'provincia_comercializadora': [],
 'número_factura': ['C5129757938'],
 'inicio_periodo': ['02/05/2012'],
 'fin_periodo': ['01/06/2012'],
 'importe_factura': ['84,77'],
 'fecha_cargo': ['06 de junio de 2012'],
 'consumo_periodo': [],
 'potencia_contratada': []}


Cargamos nuevamente el DataFrame inicial para quedarnos con su columna "json", que nos será útil para comparar resultados en el futuro.

In [ ]:
# Cargar el DataFrame
csv_path = "/content/drive/MyDrive/dataset_facturas_etiquetado_2.csv"
df = pd.read_csv(csv_path)


# Convertir las cadenas de tokens a listas
def to_list(column):
    return eval(column)

df['texto'] = df['texto'].apply(to_list)


In [ ]:
df

,id,texto,ner_tags,json,json_comprobacion,json_2
0,factura_836,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA, :, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FAC', '...","{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '...","{'nombre_cliente': ['ERMINIA CUESTA MACHADO', ...","{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '..."
1,factura_264,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...","{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '..."
2,factura_533,"[DATOS, DE, LA, FACTURA, Nº, factura, :, I3360...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NUMF', ...","{'nombre_cliente': 'Iluminada Donato Vallejo',...",{'nombre_cliente': ['Iluminada Donato Vallejo'...,"{'nombre_cliente': 'Iluminada Donato Vallejo',..."
3,factura_311,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",{'nombre_cliente': 'Celestino Estrada Santiago...,{'nombre_cliente': ['Celestino Estrada Santiag...,{'nombre_cliente': 'Celestino Estrada Santiago...
4,factura_473,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '...","{'nombre_cliente': ['Candelaria Ayala Icaza', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '..."
...,...,...,...,...,...,...
909,factura_156,"[Página, 1, /, 2, ROMA, ENERGÍAS, S.L, ., CIF,...","['O', 'O', 'O', 'O', 'B-NOMC', 'I-NOMC', 'I-NO...","{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni...","{'nombre_cliente': ['SATURIO LEYVA TAMES', 'SA...","{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni..."
910,factura_97,"[DATOS, DE, LA, FACTURA, Nº, factura, :, XR975...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NUMF', ...","{'nombre_cliente': 'Fermín Vívez Padilla', 'dn...","{'nombre_cliente': ['Fermín Vívez Padilla', 'F...","{'nombre_cliente': 'Fermín Vívez Padilla', 'dn..."
911,factura_501,"[DATOS, DE, LA, FACTURA, Nº, factura, :, LW156...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NUMF', ...","{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ...","{'nombre_cliente': ['EUSTAQUIO ARROLLO NIETO',...","{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ..."
912,factura_731,"[DATOS, DE, LA, FACTURA, Nº, factura, :, L7944...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NUMF', ...","{'nombre_cliente': 'CIRIACO LIRA PEÑA', 'dni_c...","{'nombre_cliente': ['CIRIACO LIRA PEÑA', 'CIRI...","{'nombre_cliente': 'CIRIACO LIRA PEÑA', 'dni_c..."


In [ ]:
# Selecciona las columnas deseadas del DataFrame unlabeled_df
unlabeled_df_selected = unlabeled_df[['id', 'predict', 'json_comprobacion']]

# Selecciona las columnas deseadas del DataFrame df
df_selected = df[['id', 'texto', 'json']]

# Realiza un merge horizontal utilizando la columna id como clave
merged_df = pd.merge(unlabeled_df_selected, df_selected, on='id', how='inner')

# Reorganiza el orden de las columnas
merged_df = merged_df.reindex(columns=['id', 'texto', 'predict', 'json', 'json_comprobacion'])

# Muestra el DataFrame resultante
merged_df

,id,texto,predict,json,json_comprobacion
0,factura_0,"[DATOS, DE, LA, FACTURA, Nº, factura, :, SV504...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...","{'nombre_cliente': 'Conrado Daniel Iglesias', ...",{'nombre_cliente': ['Conrado Daniel Iglesias']...
1,factura_2,"[DATOS, DE, LA, FACTURA, Nº, factura, :, H4623...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...",{'nombre_cliente': 'BENEDICTA GALLEGOS AGUILAR...,{'nombre_cliente': ['BENEDICTA GALLEGOS AGUILA...
2,factura_3,"[DATOS, DE, LA, FACTURA, Nº, factura, :, SF395...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...","{'nombre_cliente': 'Belinda Zetina Mijares', '...","{'nombre_cliente': ['Belinda Zetina Mijares'],..."
3,factura_4,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA, :, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'PANTALEÓN VELASCO DE ALBA'...,{'nombre_cliente': ['PANTALEÓN VELASCO DE ALBA...
4,factura_5,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'SATURNINO MALTÉS NARANJO',...",{'nombre_cliente': ['SATURNINO MALTÉS NARANJO'...
...,...,...,...,...,...
909,factura_994,"[Página, 1, /, 2, VIRGIN, ELECTRIC, SLU, CIF, ...","[O, O, O, O, O, B-NOMC, B-NOMC, B-NOMC, B-NOMC...","{'nombre_cliente': 'PETRONIO RAMÍREZ COLLADO',...",{'nombre_cliente': ['PETRONIO RAMÍREZ COLLADO'...
910,factura_995,"[Página, 1, /, 2, ELECTRICA, NTRA, SRA, DE, GR...","[O, O, O, O, O, B-NOMC, B-NOMC, B-NOMC, B-NOMC...","{'nombre_cliente': 'SULPICIO ESCOVAR FONSECA',...",{'nombre_cliente': ['SULPICIO ESCOVAR FONSECA'...
911,factura_996,"[DATOS, DE, LA, FACTURA, Nº, factura, :, U2093...","[O, O, O, O, O, O, O, O, O, O, O, B-NUMF, B-NU...","{'nombre_cliente': 'Petrona Uribe Naranjo', 'd...","{'nombre_cliente': ['Petrona Uribe Naranjo'], ..."
912,factura_997,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA, :, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'CELESTINA TREMINIO VALLEJO...,{'nombre_cliente': ['CELESTINA TREMINIO VALLEJ...



Como el desempeño del modelo BERT no es perfecto (falla sobre todo en los valores numéricos, algo previsible hasta cierto punto), vamos a complementar estos resultados con reglas.

In [ ]:
# Diccionario de plantilla con las claves especificadas
plantilla_reglas = {
    'nombre_cliente': [],
    'dni_cliente': [],
    'calle_cliente': [],
    'cp_cliente': [],
    'población_cliente': [],
    'provincia_cliente': [],
    'nombre_comercializadora': [],
    'cif_comercializadora': [],
    'dirección_comercializadora': [],
    'cp_comercializadora': [],
    'población_comercializadora': [],
    'provincia_comercializadora': [],
    'número_factura': [],
    'inicio_periodo': [],
    'fin_periodo': [],
    'importe_factura': [],
    'fecha_cargo': [],
    'consumo_periodo': [],
    'potencia_contratada': []
}

# Crear una nueva columna 'reglas' con el diccionario de plantilla
merged_df['reglas'] = merged_df.apply(lambda x: plantilla_reglas.copy(), axis=1)

In [ ]:
# Expresiones regulares para buscar DNIs/NIEs, CIFs, el importe y el consumo
dni_regex = r'\b(?:\d{8}[A-Z]|\d{8}-[A-Z]|[XYZ]\d{7}[A-Z]|[XYZ]-\d{7}-[A-Z]|[XYZ]\d{7}-[A-Z])\b'
cif_regex = r'\b(?:[A-HJUV]\d{8}|[A-HJUV]\d{8}-)\b'
importe_regex = r'\b(\d+[.,]?\d*)\s*(€|euros)\b'
consumo_regex = r'\b(\d+[.,]?\d*)\s*(kWh|kilovatio hora|Kilovatio hora|KILOVATIO HORA)\b'
potencia_regex = r'\b\d+(?:[,.]\d+)?\s*(?:kW|kilovatio|Kilovatio|KILOVATIO)\b'

def buscar_dni_nie(texto_lista):
    """Busca DNIs y NIEs en una lista de texto y devuelve el más común."""
    texto = " ".join(texto_lista)  # Convertir la lista de strings en una sola cadena
    coincidencias = re.findall(dni_regex, texto)
    if coincidencias:
        contador = Counter(coincidencias)
        # Devuelve el más común, en caso de empate, el primero encontrado
        return contador.most_common(1)[0][0]
    return None

def buscar_cif(texto_lista):
    """Busca CIFs en una lista de texto y devuelve el más común."""
    texto = " ".join(texto_lista)  # Convertir la lista de strings en una sola cadena
    coincidencias = re.findall(cif_regex, texto)
    if coincidencias:
        contador = Counter(coincidencias)
        # Devuelve el más común, en caso de empate, el primero encontrado
        return contador.most_common(1)[0][0]
    return None

def buscar_importe(texto_lista):
    """Busca importes en una lista de texto y devuelve el más alto."""
    texto = " ".join(texto_lista)  # Convertir la lista de strings en una sola cadena
    coincidencias = re.findall(importe_regex, texto)
    if coincidencias:
        # Convertir todas las coincidencias a números flotantes
        importes = [float(importe.replace(',', '.')) for importe, _ in coincidencias]
        # Verificar si el importe más alto es 0 euros
        if max(importes) == 0:
            return 0
        # Devolver el valor máximo
        return max(importes)
    return None  # Devolver None si no se encuentra ninguna coincidencia

def buscar_consumo(texto_lista):
    """Busca consumos en una lista de texto y devuelve la diferencia entre las dos mayores coincidencias."""
    texto = " ".join(texto_lista)  # Convertir la lista de strings en una sola cadena
    coincidencias = re.findall(consumo_regex, texto)
    if coincidencias:
        # Convertir todas las coincidencias a números flotantes
        consumos = [float(consumo.replace(',', '.')) for consumo, _ in coincidencias]
        # Tomar las dos mayores coincidencias
        dos_mayores = nlargest(2, consumos)
        # Calcular la diferencia entre las dos mayores coincidencias
        diferencia = dos_mayores[0] - dos_mayores[1]
        return diferencia if diferencia != 0 else 0  # Devolver 0 si la diferencia es 0
    return None

def buscar_potencia_contratada(texto_lista):
    texto = " ".join(texto_lista)  # Convertir la lista de strings en una sola cadena
    coincidencias = re.findall(potencia_regex, texto)
    if coincidencias:
        # Ordenar las coincidencias por longitud de cadena de mayor a menor
        coincidencias_ordenadas = sorted(coincidencias, key=len, reverse=True)
        # Tomar la coincidencia más larga
        potencia_contratada = coincidencias_ordenadas[0]
        # Si la potencia contratada es 0, devolver 0
        if potencia_contratada == '0':
            return 0
        return potencia_contratada
    return None

def actualizar_reglas(row):
    """Actualiza la columna 'reglas' con el DNI o NIE, CIF, el importe más alto, la diferencia entre los dos mayores consumos y la potencia contratada más larga encontrada en 'texto'."""
    texto_lista = row['texto']
    dni_nie_mas_comun = buscar_dni_nie(texto_lista)
    cif_mas_comun = buscar_cif(texto_lista)
    importe_mas_alto = buscar_importe(texto_lista)
    diferencia_consumo = buscar_consumo(texto_lista)
    potencia_contratada = buscar_potencia_contratada(texto_lista)

    if dni_nie_mas_comun:
        row['reglas']['dni_cliente'] = [dni_nie_mas_comun]
    if cif_mas_comun:
        row['reglas']['cif_comercializadora'] = [cif_mas_comun]
    if importe_mas_alto > 0:  # Verificar si el importe es mayor que 0
        row['reglas']['importe_factura'] = [importe_mas_alto]
    else:
        row['reglas']['importe_factura'] = [0]  # Si es 0, establecer el importe como 0
    if diferencia_consumo is not None:  # Verificar si la diferencia es None o 0
        row['reglas']['consumo_periodo'] = [diferencia_consumo]
    else:
        row['reglas']['consumo_periodo'] = [0]  # Si es None, establecer la diferencia como 0
    if potencia_contratada is not None:  # Verificar si se encontró alguna potencia contratada
        row['reglas']['potencia_contratada'] = [potencia_contratada]
    else:
        row['reglas']['potencia_contratada'] = [0]  # Si no se encuentra ninguna, establecer la potencia como 0

    return row

# Aplicar la función a cada fila del DataFrame
merged_df = merged_df.apply(actualizar_reglas, axis=1)






In [ ]:
# Expresión regular para buscar fechas en formato DD.MM.YYYY
fecha_regex = r'\b\d{2}\.\d{2}\.\d{4}\b'

def buscar_fechas(texto_lista):
    """Busca fechas en una lista de texto y devuelve una lista de objetos datetime."""
    texto = " ".join(texto_lista)  # Convertir la lista de strings en una sola cadena
    coincidencias = re.findall(fecha_regex, texto)
    fechas = []
    for coincidencia in coincidencias:
        try:
            fecha = datetime.strptime(coincidencia, '%d.%m.%Y')
            fechas.append(fecha)
        except ValueError:
            continue
    return fechas

def actualizar_fechas(row):
    """Actualiza las claves de fechas en la columna 'reglas' usando las fechas más antiguas encontradas en 'texto'."""
    texto_lista = row['texto']
    fechas = buscar_fechas(texto_lista)
    fechas_unicas = sorted(set(fechas))

    # Asignar las fechas a las claves correspondientes en 'reglas'
    if len(fechas_unicas) > 0:
        row['reglas']['inicio_periodo'] = [fechas_unicas[0].strftime('%d.%m.%Y')]
    if len(fechas_unicas) > 1:
        row['reglas']['fin_periodo'] = [fechas_unicas[1].strftime('%d.%m.%Y')]
    if len(fechas_unicas) > 2:
        row['reglas']['fecha_cargo'] = [fechas_unicas[2].strftime('%d.%m.%Y')]

    return row

# Aplicar la función a cada fila del DataFrame
merged_df = merged_df.apply(actualizar_fechas, axis=1)

In [ ]:
merged_df['reglas'][4]

{'nombre_cliente': [],
 'dni_cliente': ['13714122G'],
 'calle_cliente': [],
 'cp_cliente': [],
 'población_cliente': [],
 'provincia_cliente': [],
 'nombre_comercializadora': [],
 'cif_comercializadora': ['B24726044'],
 'dirección_comercializadora': [],
 'cp_comercializadora': [],
 'población_comercializadora': [],
 'provincia_comercializadora': [],
 'número_factura': [],
 'inicio_periodo': ['13.01.2018'],
 'fin_periodo': ['14.03.2018'],
 'importe_factura': [99.81],
 'fecha_cargo': ['16.03.2018'],
 'consumo_periodo': [246.0],
 'potencia_contratada': ['2,668 kW']}

In [ ]:
merged_df['json_comprobacion'][4]

{'nombre_cliente': ['SATURNINO MALTÉS NARANJO'],
 'dni_cliente': ['13714122G'],
 'calle_cliente': ['Calle de la Fuente de Antón Merlo'],
 'cp_cliente': ['24920'],
 'población_cliente': ['Valdepolo'],
 'provincia_cliente': [],
 'nombre_comercializadora': ['ENERVER ENERGIA, S. L.'],
 'cif_comercializadora': [],
 'dirección_comercializadora': ['CALLE 42, POLIGONO EL BONY'],
 'cp_comercializadora': ['46740'],
 'población_comercializadora': [],
 'provincia_comercializadora': ['VALENCIA'],
 'número_factura': ['KS8622022848'],
 'inicio_periodo': [],
 'fin_periodo': [],
 'importe_factura': [],
 'fecha_cargo': [],
 'consumo_periodo': [],
 'potencia_contratada': []}

In [ ]:
def actualizar_json_comprobacion(row):
    json_comprobacion = row['json_comprobacion']
    reglas = row['reglas']

    for key, value in reglas.items():
        if key in json_comprobacion and not json_comprobacion[key]:
            json_comprobacion[key] = value

    return json_comprobacion


In [ ]:
merged_df['json_comprobacion'] = merged_df.apply(actualizar_json_comprobacion, axis=1)


In [ ]:
merged_df['json_comprobacion'][0]

{'nombre_cliente': ['Conrado Daniel Iglesias'],
 'dni_cliente': ['73635161V'],
 'calle_cliente': ['Calle la Solana'],
 'cp_cliente': ['22394'],
 'población_cliente': ['La Fueva'],
 'provincia_cliente': ['Huesca'],
 'nombre_comercializadora': ['IBERDESA COMERCIALIZADORA SOCIEDAD LIMITADA'],
 'cif_comercializadora': ['B90393497'],
 'dirección_comercializadora': ['CRT.SEVILLA-MADRID, KM 524, CAMINO DE LA PASTORA S/ N'],
 'cp_comercializadora': ['41410'],
 'población_comercializadora': ['CARMONA'],
 'provincia_comercializadora': [],
 'número_factura': ['SV5043664894'],
 'inicio_periodo': ['26/08/2018'],
 'fin_periodo': ['25/09/2018'],
 'importe_factura': [19.78],
 'fecha_cargo': ['30 de septiembre de 2018'],
 'consumo_periodo': [0],
 'potencia_contratada': ['4,799 kW']}


Para los valores de localidades o provincias, creamos código para deducirlos a partir del código postal.

In [ ]:

# Cargar el DataFrame
csv_path = "/content/drive/MyDrive/codigos_postales_municipios.csv"
df_cp = pd.read_csv(csv_path)


In [ ]:
df_cp


,codigo_postal,municipio_id,municipio_nombre
0,43,43110,"Pobla de Massaluca, La"
1,85,49064,Fariza
2,633,26005,Albelda de Iregua
3,1001,1059,Vitoria-Gasteiz
4,1002,1059,Vitoria-Gasteiz
...,...,...,...
14603,52002,52001,Melilla
14604,52003,52001,Melilla
14605,52004,52001,Melilla
14606,52005,52001,Melilla


In [ ]:
# Lista de provincias con sus códigos
provincias = {
    1: 'Álava',
    2: 'Albacete',
    3: 'Alicante',
    4: 'Almería',
    5: 'Ávila',
    6: 'Badajoz',
    7: 'Baleares',
    8: 'Barcelona',
    9: 'Burgos',
    10: 'Cáceres',
    11: 'Cádiz',
    12: 'Castellón',
    13: 'Ciudad Real',
    14: 'Córdoba',
    15: 'A Coruña',
    16: 'Cuenca',
    17: 'Girona',
    18: 'Granada',
    19: 'Guadalajara',
    20: 'Guipúzcoa',
    21: 'Huelva',
    22: 'Huesca',
    23: 'Jaén',
    24: 'León',
    25: 'Lleida',
    26: 'La Rioja',
    27: 'Lugo',
    28: 'Madrid',
    29: 'Málaga',
    30: 'Murcia',
    31: 'Navarra',
    32: 'Ourense',
    33: 'Asturias',
    34: 'Palencia',
    35: 'Las Palmas',
    36: 'Pontevedra',
    37: 'Salamanca',
    38: 'Santa Cruz de Tenerife',
    39: 'Cantabria',
    40: 'Segovia',
    41: 'Sevilla',
    42: 'Soria',
    43: 'Tarragona',
    44: 'Teruel',
    45: 'Toledo',
    46: 'Valencia',
    47: 'Valladolid',
    48: 'Vizcaya',
    49: 'Zamora',
    50: 'Zaragoza',
    51: 'Ceuta',
    52: 'Melilla'
}

# Convertir la columna "codigo_postal" a tipo string
df_cp['codigo_postal'] = df_cp['codigo_postal'].astype(str)

# Añadir ceros por la izquierda para que todos los códigos postales tengan cinco dígitos
df_cp['codigo_postal'] = df_cp['codigo_postal'].apply(lambda x: x.zfill(5))

# Crear la columna "provincia" y asignar el nombre de la provincia correspondiente a cada código postal
df_cp['provincia'] = df_cp['codigo_postal'].str[:2].astype(int).map(provincias)



In [ ]:
# Función para transformar nombres de municipios
def transformar_nombre_municipio(nombre):
    if ',' in nombre:
        partes = nombre.split(', ')
        if len(partes) == 2:
            return f"{partes[1]} {partes[0]}"
    return nombre

# Aplicar la función a la columna 'municipio_nombre'
df_cp['municipio_nombre'] = df_cp['municipio_nombre'].apply(transformar_nombre_municipio)

In [ ]:
df_cp


,codigo_postal,municipio_id,municipio_nombre,provincia
0,00043,43110,La Pobla de Massaluca,NaN
1,00085,49064,Fariza,NaN
2,00633,26005,Albelda de Iregua,NaN
3,01001,1059,Vitoria-Gasteiz,Álava
4,01002,1059,Vitoria-Gasteiz,Álava
...,...,...,...,...
14603,52002,52001,Melilla,Melilla
14604,52003,52001,Melilla,Melilla
14605,52004,52001,Melilla,Melilla
14606,52005,52001,Melilla,Melilla


In [ ]:
def completar_informacion_cliente(row):
    json_comprobacion = row['json_comprobacion']

        # Verificar si 'población_cliente' está presente y está vacía
    if 'población_cliente' in json_comprobacion and not json_comprobacion['población_cliente']:
        if 'cp_cliente' in json_comprobacion and json_comprobacion['cp_cliente']:
            codigo_postal = json_comprobacion['cp_cliente'][0]
            municipio_info = df_cp[df_cp['codigo_postal'] == codigo_postal]
            if not municipio_info.empty:
                municipio = municipio_info['municipio_nombre'].iloc[0]
                json_comprobacion['población_cliente'] = [municipio]

    # Verificar si 'provincia_cliente' está presente y está vacía
    if 'provincia_cliente' in json_comprobacion and not json_comprobacion['provincia_cliente']:
        if 'cp_cliente' in json_comprobacion and json_comprobacion['cp_cliente']:
            codigo_postal = json_comprobacion['cp_cliente'][0]
            provincia_info = df_cp[df_cp['codigo_postal'] == codigo_postal]
            if not provincia_info.empty:
                provincia = provincia_info['provincia'].iloc[0]
                json_comprobacion['provincia_cliente'] = [provincia]

    # Verificar si 'población_comercializadora' está presente y está vacía
    if 'población_comercializadora' in json_comprobacion and not json_comprobacion['población_comercializadora']:
        if 'cp_comercializadora' in json_comprobacion and json_comprobacion['cp_comercializadora']:
            codigo_postal_com = json_comprobacion['cp_comercializadora'][0]
            municipio_info_com = df_cp[df_cp['codigo_postal'] == codigo_postal_com]
            if not municipio_info_com.empty:
                municipio_com = municipio_info_com['municipio_nombre'].iloc[0]
                json_comprobacion['población_comercializadora'] = [municipio_com]

    # Verificar si 'provincia_comercializadora' está presente y está vacía
    if 'provincia_comercializadora' in json_comprobacion and not json_comprobacion['provincia_comercializadora']:
        if 'cp_comercializadora' in json_comprobacion and json_comprobacion['cp_comercializadora']:
            codigo_postal_com = json_comprobacion['cp_comercializadora'][0]
            provincia_info_com = df_cp[df_cp['codigo_postal'] == codigo_postal_com]
            if not provincia_info_com.empty:
                provincia_com = provincia_info_com['provincia'].iloc[0]
                json_comprobacion['provincia_comercializadora'] = [provincia_com]

    return json_comprobacion

# Aplicar la función a cada fila del DataFrame merged_df
merged_df['json_comprobacion'] = merged_df.apply(completar_informacion_cliente, axis=1)


In [ ]:
merged_df['json_comprobacion'][25]

{'nombre_cliente': ['SIMONA NAVARRO ESCOVAR'],
 'dni_cliente': ['88560078G'],
 'calle_cliente': ['Plaza de Guadarrama'],
 'cp_cliente': ['32812'],
 'población_cliente': ['A Bola'],
 'provincia_cliente': ['Ourense'],
 'nombre_comercializadora': ['ATENCO ENERGIA SL'],
 'cif_comercializadora': ['B76366723'],
 'dirección_comercializadora': ['CALLE PORTUGAL 14, 4A'],
 'cp_comercializadora': ['35010'],
 'población_comercializadora': ['Las Palmas de Gran Canaria'],
 'provincia_comercializadora': ['PALMAS, LAS'],
 'número_factura': ['YJ3580972910'],
 'inicio_periodo': ['05.02.1999'],
 'fin_periodo': ['07.03.1999'],
 'importe_factura': [85.65],
 'fecha_cargo': ['09.03.1999'],
 'consumo_periodo': [333.0],
 'potencia_contratada': ['3,269 kW']}

In [ ]:
merged_df['json'][0]

"{'nombre_cliente': 'Conrado Daniel Iglesias', 'dni_cliente': '73635161V', 'calle_cliente': 'Calle la Solana', 'cp_cliente': '22394', 'población_cliente': 'La Fueva', 'provincia_cliente': 'Huesca', 'nombre_comercializadora': 'IBERDESA COMERCIALIZADORA SOCIEDAD LIMITADA', 'cif_comercializadora': 'B90393497', 'dirección_comercializadora': 'CRT.SEVILLA-MADRID, KM 524, CAMINO DE LA PASTORA S/N', 'cp_comercializadora': '41410', 'población_comercializadora': 'CARMONA', 'provincia_comercializadora': 'SEVILLA', 'número_factura': 'SV5043664894', 'inicio_periodo': '26.08.2018', 'fin_periodo': '25.09.2018', 'importe_factura': '19,78', 'fecha_cargo': '30.09.2018', 'consumo_periodo': 0, 'potencia_contratada': '4,799'}"

In [ ]:
# Guardar los resultados en un nuevo archivo CSV
output_path = "/content/drive/MyDrive/dataset_con_resultados.csv"
merged_df.to_csv(output_path, index=False)

Debido a que google colab no admite la configuración 'es_ES.UTF-8'
tenemos que terminar el proceso en entorno local.
Continua en el notebook 006_ejemplo_uso de la carpeta local.